In [1]:
import argparse
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from dataclasses import dataclass
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from model import GModel
from myutils import roc_auc, translate_result, filter_target
from load_data import load_data
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
@dataclass
class Args:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    data: str = "gdsc1"
    n_jobs: int = 15 # ← ここをコア数に応じて調整
    lr: float = 5e-4
    epochs: int = 1000

In [4]:
args = Args()
target_option = "cell"  # "cell" か "drug"

In [5]:
# データ読み込み
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

load gdsc1
response matrix (res) shape: (300, 925)
exprs shape: (1084, 19562)
mut shape: (1084, 18099)
cna shape: (1084, 24502)
925
exprs shape: (925, 19562)
mut shape: (925, 18099)
cna shape: (925, 24502)
drug_feature shape: (300, 920)
response matrix (res) shape: (925, 300)
null_mask shape: (925, 300)


In [6]:
def run_single_model(exprs, cna, mut, drug_feature, res_mat, null_mask, target_dim, target_index, args, seed):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    val_labels = sampler.test_data[sampler.test_mask]

    model = GModel(
        adj_mat=sampler.train_data.float(),
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=args.device,
    )
    opt = Optimizer(
        model=model,
        train_data=sampler.train_data,
        test_data=sampler.test_data,
        test_mask=sampler.test_mask,
        train_mask=sampler.train_mask,
        evaluate_fun=roc_auc,
        lr=args.lr,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    _, true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
# ターゲット次元と統計
target_dim = 0 if target_option == "cell" else 1
samples = res.shape[target_dim]
cell_sum = np.sum(res.values, axis=1)
drug_sum = np.sum(res.values, axis=0)

In [8]:
# 並列実行用ラッパー
def process_target(seed, target_index):
    try:
        return run_single_model(
            exprs=exprs,
            cna=cna,
            mut=mut,
            drug_feature=drug_feature,
            res_mat=res.values,
            null_mask=null_mask,
            target_dim=target_dim,
            target_index=target_index,
            args=args,
            seed=seed,
        )
    except Exception as e:
        print(f"❌ Failed at target {target_index}: {e}")
        return None

In [9]:
# 結果格納用
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
skipped_targets = []
passed_targets = []

# スキップチェック
for target_index in range(samples):
    label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
    passed, reason, pos, neg, total = filter_target(label_vec)

    if passed:
        passed_targets.append(target_index)
    else:
        skipped_targets.append((target_index, reason, pos, neg, total))

# スキップ情報表示
print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
for idx, reason, pos, neg, total in skipped_targets:
    print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")

# 並列実行（max_njobs を args.n_jobs に変更可能）
results = Parallel(n_jobs=args.n_jobs)(
    delayed(process_target)(seed, target_index)
    for seed, target_index in enumerate(tqdm(passed_targets, desc=f"MOFGCN ({args.data} - {target_option})"))
)

# 結果の統合（None を除外）
for r in results:
    if r is not None:
        true_data, pred_data = r
        true_data_s = pd.concat([true_data_s, translate_result(true_data)], ignore_index=True)
        predict_data_s = pd.concat([predict_data_s, translate_result(pred_data)], ignore_index=True)


🚫 Skipped Targets: 10
Target 182: skipped because few_total_samples (total=5, pos=3, neg=2)
Target 253: skipped because few_total_samples (total=8, pos=2, neg=6)
Target 267: skipped because few_total_samples (total=9, pos=2, neg=7)
Target 377: skipped because few_total_samples (total=8, pos=7, neg=1)
Target 483: skipped because low_negative_ratio (total=12, pos=12, neg=0)
Target 492: skipped because few_total_samples (total=8, pos=2, neg=6)
Target 502: skipped because few_total_samples (total=8, pos=7, neg=1)
Target 773: skipped because low_negative_ratio (total=79, pos=79, neg=0)
Target 829: skipped because low_negative_ratio (total=78, pos=77, neg=1)
Target 908: skipped because few_total_samples (total=6, pos=2, neg=4)


MOFGCN (gdsc1 - cell):   2%|▏         | 15/915 [00:00<00:19, 47.12it/s]/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  gene = torch.from_numpy(gene).to(device)
/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered interna

epoch:   0 loss:0.738288 auc:0.6000
epoch:  20 loss:0.675807 auc:0.8636
epoch:  40 loss:0.632773 auc:0.8636
epoch:  60 loss:0.571758 auc:0.8364
epoch:  80 loss:0.519814 auc:0.8091
epoch: 100 loss:0.507696 auc:0.8091
epoch: 120 loss:0.499700 auc:0.7727
epoch: 140 loss:0.494410 auc:0.8182
epoch: 160 loss:0.489212 auc:0.8364
epoch: 180 loss:0.492077 auc:0.8455
Fit finished.
epoch:   0 loss:0.787209 auc:0.5833
epoch:  20 loss:0.671509 auc:0.9167
epoch:  40 loss:0.624449 auc:0.8333
epoch:  60 loss:0.557344 auc:0.8333
epoch:  80 loss:0.518290 auc:1.0000
epoch: 100 loss:0.504305 auc:1.0000
epoch: 120 loss:0.504941 auc:1.0000
epoch: 140 loss:0.495327 auc:1.0000
epoch: 160 loss:0.491979 auc:1.0000
epoch: 180 loss:0.492066 auc:1.0000
epoch: 200 loss:0.488823 auc:1.0000
epoch: 220 loss:0.485438 auc:1.0000
epoch: 240 loss:0.515239 auc:1.0000
epoch: 260 loss:0.481866 auc:1.0000
epoch: 280 loss:0.469759 auc:1.0000
epoch: 300 loss:0.448355 auc:1.0000
epoch: 320 loss:0.420741 auc:1.0000
epoch: 340 los

MOFGCN (gdsc1 - cell):  28%|██▊       | 255/915 [01:32<03:44,  2.94it/s]

epoch:   0 loss:0.730150 auc:0.5476
epoch:  20 loss:0.668324 auc:0.5556
epoch:  40 loss:0.631200 auc:0.6270
epoch:  60 loss:0.556668 auc:0.5952
epoch:  80 loss:0.517413 auc:0.5952
epoch: 100 loss:0.504413 auc:0.6429
epoch: 120 loss:0.496658 auc:0.6587
epoch: 140 loss:0.492999 auc:0.6190
epoch: 160 loss:0.486660 auc:0.6508
epoch: 180 loss:0.479780 auc:0.7302
epoch: 200 loss:0.466290 auc:0.7302
epoch: 220 loss:0.458040 auc:0.7143
epoch: 240 loss:0.436026 auc:0.7222
epoch: 260 loss:0.418716 auc:0.7698
epoch: 280 loss:0.390648 auc:0.7778
epoch: 300 loss:0.373744 auc:0.7619
epoch: 320 loss:0.339381 auc:0.6746
epoch: 340 loss:0.310066 auc:0.6825
Fit finished.
epoch:   0 loss:1.014029 auc:0.4526
epoch:  20 loss:0.674688 auc:0.6536
epoch:  40 loss:0.646573 auc:0.6405
epoch:  60 loss:0.610365 auc:0.6291
epoch:  80 loss:0.550221 auc:0.6291
epoch: 100 loss:0.514597 auc:0.6324
epoch: 120 loss:0.504087 auc:0.6356
epoch: 140 loss:0.498864 auc:0.6536
epoch: 160 loss:0.493319 auc:0.6389
epoch: 180 los

...

epoch: 160 loss:0.489405 auc:0.5032
Fit finished.
epoch:   0 loss:0.804228 auc:0.3929
epoch:  20 loss:0.652137 auc:0.5357
epoch:  40 loss:0.603589 auc:0.6339
epoch:  60 loss:0.530110 auc:0.7143
epoch:  80 loss:0.510258 auc:0.6964
epoch: 100 loss:0.518114 auc:0.6518
epoch: 120 loss:0.497886 auc:0.6429
epoch: 140 loss:0.496470 auc:0.6429
epoch: 160 loss:0.488952 auc:0.6161
epoch: 180 loss:0.485028 auc:0.5982
epoch: 200 loss:0.474587 auc:0.5714
Fit finished.
epoch:   0 loss:0.707964 auc:0.2143
epoch:  20 loss:0.627679 auc:0.7857
epoch:  40 loss:0.533266 auc:0.8929
epoch:  60 loss:0.515046 auc:0.8214
epoch:  80 loss:0.501968 auc:0.7857
epoch: 100 loss:0.494795 auc:0.8214
epoch: 120 loss:0.490173 auc:0.7857
epoch: 140 loss:0.475094 auc:0.6429
epoch: 160 loss:0.454731 auc:0.6429
epoch: 180 loss:0.435458 auc:0.5714
Fit finished.
epoch:   0 loss:0.834317 auc:0.1379
epoch:  20 loss:0.661738 auc:0.7586
epoch:  40 loss:0.628557 auc:0.7931
epoch:  60 loss:0.562633 auc:0.8621
epoch:  80 loss:0.5170

epoch:  40 loss:0.641054 loss:0.504372 auc:0.3636
epoch: 140 loss:0.495288 auc:0.3333
epoch: 160 loss:0.492512 auc:0.3333
Fit finished.
epoch:   0 loss:0.734662 auc:0.1250
epoch:  20 loss:0.676982 auc:0.5000
epoch:  40 loss:0.636895 auc:0.5417
epoch:  60 loss:0.584984 auc:0.6250
epoch:  80 loss:0.527889 auc:0.6250
epoch: 100 loss:0.509666 auc:0.7083
epoch: 120 loss:0.500014 auc:0.6667
epoch: 140 loss:0.494793 auc:0.6667
epoch: 160 loss:0.494176 auc:0.7500
epoch: 180 loss:0.481630 auc:0.9167
epoch: 200 loss:0.477854 auc:0.9167
epoch: 220 loss:0.457970 auc:0.9583
epoch: 240 loss:0.447854 auc:0.7917
epoch: 260 loss:0.414138 auc:0.6667
Fit finished.
epoch:   0 loss:0.729629 auc:0.4576
epoch:  20 loss:0.658422 auc:0.4061
epoch:  40 loss:0.621945 auc:0.4909
epoch:  60 loss:0.537584 auc:0.4879
epoch:  80 loss:0.514986 auc:0.4606
epoch: 100 loss:0.506054 auc:0.4273
epoch: 120 loss:0.495035 auc:0.4303
epoch: 140 loss:0.490234 auc:0.4848
epoch: 160 loss:0.486008 auc:0.5697
epoch: 180 loss:0.4935

epoch: 160 loss:0.491888 auc:0.7955
epoch: 180 loss:0.483434 auc:0.8500
epoch: 200 loss:0.477815 auc:0.8364
epoch: 220 loss:0.468181 auc:0.8045
epoch: 240 loss:0.448762 auc:0.7227
epoch: 260 loss:0.433398 auc:0.7273
Fit finished.
epoch:   0 loss:0.758450 auc:0.3796
epoch:  20 loss:0.657873 auc:0.5185
epoch:  40 loss:0.612279 auc:0.4630
epoch:  60 loss:0.528736 auc:0.5000
epoch:  80 loss:0.512502 auc:0.4815
epoch: 100 loss:0.501642 auc:0.5093
epoch: 120 loss:0.495660 auc:0.5093
epoch: 140 loss:0.491916 auc:0.5185
epoch: 160 loss:0.494098 auc:0.4907
epoch: 180 loss:0.484156 auc:0.4537
Fit finished.
epoch:   0 loss:0.770157 auc:0.4035
epoch:  20 loss:0.674275 auc:0.6140
epoch:  40 loss:0.637334 auc:0.4912
epoch:  60 loss:0.577252 auc:0.4561
epoch:  80 loss:0.523180 auc:0.4561
epoch: 100 loss:0.507772 auc:0.5614
epoch: 120 loss:0.498858 auc:0.5965
epoch: 140 loss:0.496162 auc:0.7368
epoch: 160 loss:0.490443 auc:0.7193
epoch: 180 loss:0.520925 auc:0.7895
epoch: 200 loss:0.480743 auc:0.7895


loss:0.536025 auc:0.6516
epoch: 100 loss:0.511856 auc:0.6645
epoch: 120 loss:0.503835 auc:0.6581
epoch: 140 loss:0.494925 auc:0.6839
epoch: 160 loss:0.488859 auc:0.7290
epoch: 180 loss:0.480786 auc:0.7677
epoch: 200 loss:0.487220 auc:0.7419
epoch: 220 loss:0.457946 auc:0.7355
epoch: 240 loss:0.434392 auc:0.7742
epoch: 260 loss:0.426474 auc:0.7355
epoch: 280 loss:0.394321 auc:0.6968
epoch: 300 loss:0.363027 auc:0.5871
Fit finished.
epoch:   0 loss:0.769926 auc:0.5393
epoch:  20 loss:0.667603 auc:0.5207
epoch:  40 loss:0.636740 auc:0.5723
epoch:  60 loss:0.575779 auc:0.5661
epoch:  80 loss:0.520172 auc:0.5971
epoch: 100 loss:0.503832 auc:0.6095
epoch: 120 loss:0.499411 auc:0.5971
epoch: 140 loss:0.492848 auc:0.5971
epoch: 160 loss:0.488690 auc:0.6136
epoch: 180 loss:0.484039 auc:0.6240
epoch: 200 loss:0.482727 auc:0.7293
epoch: 220 loss:0.467665 auc:0.7686
epoch: 240 loss:0.441083 auc:0.7872
epoch: 260 loss:0.417402 auc:0.7376
epoch: 280 loss:0.380823 auc:0.7665
epoch: 300 loss:0.358977 

MOFGCN (gdsc1 - cell): 100%|██████████| 915/915 [05:28<00:00,  2.79it/s]


In [10]:
# Save
true_path = f"mofgcn_true_{args.data}_{target_option}.csv"
pred_path = f"mofgcn_pred_{args.data}_{target_option}.csv"
true_data_s.to_csv(true_path, index=False)
predict_data_s.to_csv(pred_path, index=False)

print(f"\n✅ Done. Results saved to:\n  - {true_path}\n  - {pred_path}")


✅ Done. Results saved to:
  - mofgcn_true_gdsc1_cell.csv
  - mofgcn_pred_gdsc1_cell.csv
